In [33]:
import pandas as pd
#Snorkel
from snorkel.labeling import LabelingFunction
import re
from snorkel.preprocess import preprocessor
from textblob import TextBlob
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel
from snorkel.labeling import LFAnalysis
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.labeling import labeling_function
import nltk
import nltk.tokenize
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import numpy as np
punc = string.punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
nltk.download('stopwords')
from tqdm import tqdm_notebook as tqdm
stop_words = set(stopwords.words('english'))
import nltk
import nltk.tokenize
import spacy

#Tensorflow and Keras ML model libraries
import tensorflow as tf
#from tensorflow import keras
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('../../india-news-headlines.csv')
df.head()

,publish_date,headline_category,headline_text
0,20010102,unknown,Status quo will not be disturbed at Ayodhya; s...
1,20010102,unknown,Fissures in Hurriyat over Pak visit
2,20010102,unknown,America's unwanted heading for India?
3,20010102,unknown,For bigwigs; it is destination Goa
4,20010102,unknown,Extra buses to clear tourist traffic


In [3]:
df.shape

(3650970, 3)

In [4]:
df['headline_category'].unique()

array(['unknown', 'entertainment.hindi.bollywood', 'india', ...,
       'sports.football.euro-2021', 'business.markets.ipo',
       'sports.tokyo-olympics.india-in-tokyo'], dtype=object)

In [5]:
### conduct an NLTK analysis on the mostp popular words in the dataset to categorise if they are positive or negative words.
### additionally categorise by sports category

In [6]:
df = df.drop(['publish_date', 'headline_category'], axis=1)
df.head()


,headline_text
0,Status quo will not be disturbed at Ayodhya; s...
1,Fissures in Hurriyat over Pak visit
2,America's unwanted heading for India?
3,For bigwigs; it is destination Goa
4,Extra buses to clear tourist traffic


Using Snorkel for the labelling of headlines with classes of positive and negative

In [7]:
pos = 1
neg = 0
#neural
neu = -1

# function that inputs words to represent a label
def keyword_lookup(x, keywords, label):
    if any(word in x.headline_text.lower() for word in keywords):
        return label
    return neu

#function for correctly assigning a label
def make_keyword(keywords, label):
    return LabelingFunction(
        name=f"keyword_{keywords[0]}",
        f=keyword_lookup,
        resources=dict(keywords=keywords, label=label)) 

In [20]:
# I asked chat GPT to produce words for both positive and negative words that may be found within an article

# Positive words
# positive_words = ["inspiring","motivating","encouraging","empowering","enriching","fulfilling","joyful","uplifting","heartwarming",
#     "exciting","optimistic","radiant","hopeful","refreshing","transformative","grateful","compassionate","respectful","honest",
#     "supportive","friendly","amicable","cheerful","energetic","passionate","confident","creative","enthusiastic","dynamic","courageous",
#     "adventurous","fearless","bold","brave","determined","ambitious","successful","accomplished","triumphant","victorious","celebrated",
#     "esteemed","honored","proud","satisfied","content","joyous","blissful","ecstatic","radiant","gracious","generous","kind","compassionate",
#     "understanding","accepting","appreciative","respectful","patient","forgiving","calm","serene","tranquil","relaxed","peaceful",
#     "harmonious","balanced","healthy","vibrant","invigorated","rejuvenated","energetic","alive","connected","engaged","inspirational",
#     "uplifting","motivational","encouraging","supportive","nurturing","affirming","reassuring","hopeful","optimistic","confident",
#     "secure","empowered","enlightened","aware","mindful","grateful","thankful","appreciative"]

positive_words = ['achieve', 'advantage', 'affirm', 'aid', 'aim', 'alive', 'amazing', 'approve', 'ascend', 'assure', 
                  'astonishing', 'attract', 'award', 'benefit', 'best', 'better', 'bliss', 'boost', 'brave', 'breakthrough', 
                  'bright', 'brilliant', 'celebrate', 'champion', 'cheer', 'clever', 'commend', 'complete', 'congratulate', 
                  'conquer', 'convince', 'courage', 'creative', 'cure', 'dazzling', 'delight', 'deliver', 'deserve', 'destiny', 
                  'discover', 'ease', 'effective', 'efficient', 'elated', 'empower', 'encourage', 'enjoy', 'enlighten', 'enterprising', 
                  'enthusiasm', 'excellent', 'excite', 'exhilarating', 'expand', 'explore', 'fabulous', 'fair', 'famous', 
                  'fantastic', 'fascinating', 'favor', 'flourish', 'fortunate', 'free', 'fresh', 'friendly', 'fulfill', 'fun', 
                  'gain', 'generous', 'gift', 'glad', 'glamorous', 'glory', 'good', 'gorgeous', 'grace', 'gracious', 'grand', 
                  'grateful', 'great', 'grow', 'handsome', 'happy', 'harmony', 'heal', 'healthy', 'heavenly', 'helpful', 
                  'hero', 'high', 'honor', 'hope', 'hot', 'hug', 'illuminate', 'imagine', 'improve', 'incredible', 'innovate', 
                  'inspire', 'intelligent', 'invent', 'joy', 'jubilant', 'justice', 'keen', 'kind', 'laugh', 'lavish', 
                  'lead', 'legendary', 'liberate', 'light', 'like', 'lionhearted', 'lively', 'love', 'loyal', 'lucky', 'magic', 
                  'magnificent', 'marvelous', 'master', 'medal', 'mend', 'merit', 'miracle', 'motivate', 'natural', 'nice', 
                  'noble', 'nourish', 'optimal', 'outstanding', 'overcome', 'paradise', 'passion', 'peace', 'perfect', 
                  'pleasant', 'pleased', 'pleasure', 'popular', 'positive', 'power', 'praise', 'precious', 'premium', 'pretty', 
                  'pride', 'prize', 'prosper', 'proud', 'quality', 'quick', 'radiant', 'ready', 'reassure', 'recharge', 
                  'refresh', 'rejoice', 'relax', 'relief', 'remarkable', 'renew', 'renowned', 'rescue', 'resilient', 
                  'restore', 'revel', 'reward', 'rich', 'right', 'robust', 'satisfy', 'secure', 'shine', 'smile', 'smooth', 
                  'soulful', 'spark', 'spectacular', 'splendid', 'sprout', 'stellar', 'strength']

# Negative words
# negative_words = ["disappointing","frustrating","discouraging","upsetting","unfortunate","sad","heartbreaking",
#     "tragic","painful","hurtful","offensive","insulting","insensitive","harmful","damaging","destructive","terrible","horrible",
#     "awful","miserable","depressing","desperate","hopeless","dismal","gloomy","melancholy", "sadness","sorrow","anguish","agony",
#     "misery","pessimistic","cynical","skeptical","suspicious","disbelieving","doubtful","unbelievable","displeased","unsatisfied",
#     "unhappy","dissatisfied","angry","furious","enraged","infuriated","annoyed","irritated","frustrated","exasperated","stressed",
#     "Anxious","Nervous","Tense","Worried","Fearful","Afraid","Scared","Terrified","Panicked","Despair","Despondent","Defeated",
#     "Surrendered","Powerless","Inferior","Inadequate","Unimportant","Worthless","Insignificant","Unworthy","Ashamed","Embarrassed",
#     "Guilty","Remorseful","Regretful","Mortified","Humiliated","Disgusted","Repulsed","Revolted","Nauseated","Appalled","Horrified",
#     "Offended","Resentful","Bitter","Hostile","Vengeful","Hateful","Malicious", "War", 'Killed', 'Death', ]
negative_words = ['accused', 'attack', 'ban', 'bankrupt', 'beware', 'blame', 'bleak', 'block', 'bloodbath', 'bomb', 
                  'breakdown', 'bribe', 'burden', 'bust', 'catastrophe', 'chaos', 'charge', 'clash', 'collapse', 'concerns',
                  'controversy', 'crackdown', 'crisis', 'crumble', 'cut', 'damage', 'death', 'debt', 'defeat', 'deficit',
                  'delay', 'demand', 'demonstrate', 'denounce', 'depressed', 'desperate', 'deteriorating', 'disagree', 'disappoint',
                  'disarray', 'disastrous', 'disclose', 'dismiss', 'dispute', 'dissolve', 'disturbance', 'doubt', 'downturn', 
                  'drag', 'drop', 'embarrassment', 'emergency', 'endanger', 'entangled', 'evict', 'expose', 'fail', 'fall',
                  'fault', 'fear', 'fiasco', 'fight', 'fire', 'flaw', 'flee', 'flood', 'forced', 'fraud', 'freeze', 'frustrate',
                  'fury', 'harm', 'havoc', 'hit', 'horrify', 'hostile', 'hurt', 'hype', 'implode', 'inflation', 'injure', 'insult',
                  'interfere', 'interruption', 'invade', 'investigation', 'jeopardy', 'kick', 'kill', 'lawsuit', 'leak', 'lose',
                  'meltdown', 'mistake', 'nervous', 'nuisance', 'outrage', 'overrule', 'pain', 'panic', 'penalty', 'peril',
                  'poison', 'pollute', 'pressure', 'probe', 'punish', 'quarrel', 'question', 'quit', 'rally', 'rampage', 'reject',
                  'resign', 'restrict', 'retaliate', 'retreat', 'revolt', 'ridicule', 'rip', 'risk', 'rough', 'ruin', 'rupture', 
                  'scandal', 'scold', 'scorch', 'scorn', 'scream', 'seize', 'sever', 'shame', 'shatter', 'shock', 'shutdown',
                  'slump', 'smash', 'smuggle', 'spoil', 'spurn', 'stain', 'stall', 'strike', 'struggle', 'sue', 'suffer',   
                  'suspend', 'taint', 'terminate', 'terrorize', 'threat', 'thwart', 'torture', 'tragedy', 'trap', 'trespass', 
                  'trouble', 'unrest', 'upheaval', 'upset', 'vandalize', 'violate', 'warning', 'weaken', 'worsen', 'wound']




In [21]:
len(positive_words)

187

In [22]:
len(negative_words)

166

In [23]:
#  I will now add these words to the make keyword function
positive_keyword = make_keyword(keywords=positive_words, label=pos)
negative_keyword = make_keyword(keywords=negative_words, label=neg)


Will use the TextBlob library to determine the polarity and subjectivity scores

In [24]:
#
@preprocessor(memoize=True)
def textblob_sentiment(x):
    scores = TextBlob(x.headline_text)
    x.polarity = scores.sentiment.polarity
    x.subjectivity = scores.sentiment.subjectivity
    return x
#find polarity
@labeling_function(pre=[textblob_sentiment])
def textblob_polarity(x):
    return pos if x.polarity > 0.6 else neu
    # if x.polarity > 0.1:
    #     return pos
    # elif x.polarity < -0.1:
    #     return neg
    # else:
    #     return neu
#find subjectivity 
@labeling_function(pre=[textblob_sentiment])
def textblob_subjectivity(x):
    return pos if x.subjectivity >= 0.5 else neu

In [27]:
#combine all the labeling functions 
lfs = [positive_keyword, negative_keyword, textblob_polarity, textblob_subjectivity]
#apply the lfs on the dataframe
applier = PandasLFApplier(lfs=lfs)
L_snorkel = applier.apply(df=df)
#apply the label model
label_model = LabelModel(cardinality=2, verbose=True)
#fit on the data
label_model.fit(L_snorkel)
#predict and create the labels
df["label"] = label_model.predict(L=L_snorkel)

100%|██████████| 3650970/3650970 [5:16:59<00:00, 191.96it/s]   
INFO:root:Computing O...
INFO:root:Estimating \mu...
  1%|          | 1/100 [00:00<00:17,  5.55epoch/s]INFO:root:[10 epochs]: TRAIN:[loss=0.009]
INFO:root:[20 epochs]: TRAIN:[loss=0.001]
 23%|██▎       | 23/100 [00:00<00:00, 99.71epoch/s]INFO:root:[30 epochs]: TRAIN:[loss=0.002]
INFO:root:[40 epochs]: TRAIN:[loss=0.001]
 45%|████▌     | 45/100 [00:00<00:00, 145.45epoch/s]INFO:root:[50 epochs]: TRAIN:[loss=0.001]
INFO:root:[60 epochs]: TRAIN:[loss=0.001]
INFO:root:[70 epochs]: TRAIN:[loss=0.001]
 71%|███████   | 71/100 [00:00<00:00, 179.09epoch/s]INFO:root:[80 epochs]: TRAIN:[loss=0.001]
INFO:root:[90 epochs]: TRAIN:[loss=0.001]
100%|██████████| 100/100 [00:00<00:00, 161.79epoch/s]
INFO:root:Finished Training


In [30]:
df['label'].value_counts()

-1    1861021
 1    1162856
 0     627093
Name: label, dtype: int64

In [31]:
#Filtering out unlabeled data points
df= df.loc[df.label.isin([0,1]), :]
df['label'].value_counts()

1    1162856
0     627093
Name: label, dtype: int64

## Modelling
For the modelling side of the project I will use Tensor Flow and Keras to produce a Neural Network.

# NOTE: WORKING ON THIS STAGE ONWARDS, BELOW IS NOT MY FINAL CODE, JUST TESTING THIS CODE OUT

Firstly, I will train and test split the data

In [35]:
#make a copy of the dataframe
data = df.copy()
#define a function which handles the text preprocessing 
def preparation_text_data(data):
    """
    This pipeline prepares the text data, conducting the following steps:
    1) Tokenization
    2) Lemmatization
    4) Removal of stopwords
    5) Removal of punctuation
    """
    # initialize spacy object
    nlp = spacy.load('en_core_web_sm')
    # select raw text
    raw_text = data.headline_text.values.tolist()
    # tokenize
    tokenized_text = [[nlp(i.lower().strip())] for i in tqdm(raw_text)]
    #define the punctuations and stop words
    punc = string.punctuation 
    stop_words = set(stopwords.words('english'))
    #lemmatize, remove stopwords and punctuationd
    corpus = []
    for doc in tqdm(tokenized_text):
        corpus.append([word.lemma_ for word in doc[0] if (word.lemma_ not in stop_words and word.lemma_ not in punc)])
    # add prepared data to df
    data["headline_text"] = corpus
    return data
#apply the data preprocessing function
data =  preparation_text_data(data)

C:\Users\Chris\AppData\Local\Temp\ipykernel_8828\1869780026.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tokenized_text = [[nlp(i.lower().strip())] for i in tqdm(raw_text)]


  0%|          | 0/1789949 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
def text_representation(data):
  tfidf_vect = TfidfVectorizer()
  data['headline_text'] = data['headline_text'].apply(lambda text: " ".join(set(text)))
  X_tfidf = tfidf_vect.fit_transform(data['headline_text'])
  print(X_tfidf.shape)
  print(tfidf_vect.get_feature_names())
  X_tfidf = pd.DataFrame(X_tfidf.toarray())
  return X_tfidf
#apply the TFIDV function
X_tfidf = text_representation(data)

In [ ]:
X= X_tfidf
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#fit Log Regression Model
clf= LogisticRegression()
clf.fit(X_train,y_train)
clf.score(X_test,y_test)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
new_data = ["The US imposes sanctions on Rassia because of the Ukranian war"]
tf = TfidfVectorizer()
tfdf = tf.fit_transform(data['text'])
vect = pd.DataFrame(tf.transform(new_data).toarray())
new_data = pd.DataFrame(vect)
logistic_prediction = clf.predict(new_data)
print(logistic_prediction)